In [1]:
# importing Libraries 

!pip install requests beautifulsoup4 pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd

# defining a Function to scrape a single page

def scrape_page(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    quotes = []
    for quote_tag in soup.find_all('div', class_='quote'):
        quote_text = quote_tag.find('span', class_='text').text.strip()[1:-1] # Extract and remove quotes
        author = quote_tag.find('small', class_='author').text.strip()
        tags = [tag.text for tag in quote_tag.find_all('a', class_='tag')]
        quotes.append({'author': author, 'quote': quote_text, 'tags': tags})
    return quotes
    

# Scrape all pages


all_quotes = []
base_url = 'http://quotes.toscrape.com/'
page_url = base_url

while True:
    page_quotes = scrape_page(page_url)
    all_quotes.extend(page_quotes)

    next_page = BeautifulSoup(requests.get(page_url).content, 'html.parser').find('li', class_='next')
    if next_page:
        page_url = base_url + next_page.a['href']
    else:
        break

# Create a DataFrame and save to CSV

df = pd.DataFrame(all_quotes)

# Explode the 'tags' column to have one tag per row

df = df.explode('tags', ignore_index=True)
df.to_csv('quotes.csv', index=False)
print("Data saved to quotes.csv")

Data saved to quotes.csv
